# 1. Building a Review-Based QA System

In [57]:
from datasets import get_dataset_config_names

In [2]:
domain = get_dataset_config_names("subjqa")

In [4]:
domain

['books', 'electronics', 'grocery', 'movies', 'restaurants', 'tripadvisor']

In [5]:
from datasets import load_dataset

In [6]:
subjqa = load_dataset("subjqa", name="electronics")

Generating train split:   0%|          | 0/1295 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/358 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/255 [00:00<?, ? examples/s]

Dataset subjqa downloaded and prepared to /root/.cache/huggingface/datasets/subjqa/electronics/1.1.0/e5588f9298ff2d70686a00cc377e4bdccf4e32287459e3c6baf2dc5ab57fe7fd. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
subjqa["train"]["answers"][1]

{'text': ['Bass is weak as expected',
  'Bass is weak as expected, even with EQ adjusted up'],
 'answer_start': [1302, 1302],
 'answer_subj_level': [1, 1],
 'ans_subj_score': [0.5083333253860474, 0.5083333253860474],
 'is_ans_subjective': [True, True]}

In [13]:
import pandas as pd

In [28]:
subjqa.items()

dict_items([('train', Dataset({
    features: ['domain', 'nn_mod', 'nn_asp', 'query_mod', 'query_asp', 'q_reviews_id', 'question_subj_level', 'ques_subj_score', 'is_ques_subjective', 'review_id', 'id', 'title', 'context', 'question', 'answers'],
    num_rows: 1295
})), ('test', Dataset({
    features: ['domain', 'nn_mod', 'nn_asp', 'query_mod', 'query_asp', 'q_reviews_id', 'question_subj_level', 'ques_subj_score', 'is_ques_subjective', 'review_id', 'id', 'title', 'context', 'question', 'answers'],
    num_rows: 358
})), ('validation', Dataset({
    features: ['domain', 'nn_mod', 'nn_asp', 'query_mod', 'query_asp', 'q_reviews_id', 'question_subj_level', 'ques_subj_score', 'is_ques_subjective', 'review_id', 'id', 'title', 'context', 'question', 'answers'],
    num_rows: 255
}))])

In [30]:
subjqa.flatten().items()

dict_items([('train', Dataset({
    features: ['domain', 'nn_mod', 'nn_asp', 'query_mod', 'query_asp', 'q_reviews_id', 'question_subj_level', 'ques_subj_score', 'is_ques_subjective', 'review_id', 'id', 'title', 'context', 'question', 'answers.text', 'answers.answer_start', 'answers.answer_subj_level', 'answers.ans_subj_score', 'answers.is_ans_subjective'],
    num_rows: 1295
})), ('test', Dataset({
    features: ['domain', 'nn_mod', 'nn_asp', 'query_mod', 'query_asp', 'q_reviews_id', 'question_subj_level', 'ques_subj_score', 'is_ques_subjective', 'review_id', 'id', 'title', 'context', 'question', 'answers.text', 'answers.answer_start', 'answers.answer_subj_level', 'answers.ans_subj_score', 'answers.is_ans_subjective'],
    num_rows: 358
})), ('validation', Dataset({
    features: ['domain', 'nn_mod', 'nn_asp', 'query_mod', 'query_asp', 'q_reviews_id', 'question_subj_level', 'ques_subj_score', 'is_ques_subjective', 'review_id', 'id', 'title', 'context', 'question', 'answers.text', 'answ

In [18]:
dset_flatten = subjqa.flatten().items()

In [19]:
dfs = {split: dset.to_pandas() for split, dset in dset_flatten}

In [24]:
dfs

,domain,nn_mod,nn_asp,query_mod,query_asp,q_reviews_id,question_subj_level,ques_subj_score,is_ques_subjective,review_id,id,title,context,question,answers.text,answers.answer_start,answers.answer_subj_level,answers.ans_subj_score,answers.is_ans_subjective
0,electronics,great,bass response,excellent,bass,0514ee34b672623dff659334a25b599b,5,0.50,False,882b1e2745a4779c8f17b3d4406b91c7,2543d296da9766d8d17d040ecc781699,B00001P4ZH,"I have had Koss headphones in the past, Pro 4A...",How is the bass?,[],[],[],[],[]
1,electronics,harsh,high,not strong,bass,7c46670208f7bf5497480fbdbb44561a,1,0.50,False,ce76793f036494eabe07b33a9a67288a,d476830bf9282e2b9033e2bb44bbb995,B00001P4ZH,To anyone who hasn't tried all the various typ...,Is this music song have a goo bass?,"[Bass is weak as expected, Bass is weak as exp...","[1302, 1302]","[1, 1]","[0.5083333, 0.5083333]","[True, True]"
2,electronics,neutral,sound,present,bass,8fbf26792c438aa83178c2d507af5d77,1,0.50,False,d040f2713caa2aff0ce95affb40e12c2,455575557886d6dfeea5aa19577e5de4,B00001P4ZH,I have had many sub-$100 headphones from $5 Pa...,How is the bass?,[The only fault in the sound is the bass],[650],[2],[0.6333333],[True]
3,electronics,muddy,bass,awesome,bass,9876fd06ed8f075fcad70d1e30e7e8be,1,0.50,False,043e7162df91f6ea916c790c8a6f6b22,6895a59b470d8feee0f39da6c53a92e5,B00001WRSJ,My sister's Bose headphones finally died and s...,How is the audio bass?,[the best of all of them],[1609],[1],[0.3],[False]
4,electronics,perfect,bass,incredible,sound,16506b53e2d4c2b6a65881d9462256c2,1,0.65,True,29ccd7e690050e2951be49289e915382,7a2173c502da97c5bd5950eae7cd7430,B00001WRSJ,Wow. Just wow. I'm a 22 yr old with a crazy ob...,Why do I have an incredible sound?,"[The sound is so crisp, crazy obsession with s...","[141, 38]","[1, 1]","[0.40833333, 0.40833333]","[False, False]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1290,electronics,great,impression,rubbery,feel,d4409d6272a7d1d5aa287d6e164bb7d0,1,0.00,False,ebaaec25661c0b21f3e5600a1ed7e30b,e0ea72e711324b5482ec1b36a48f16ee,B00H3OYSHW,I ordered my new NewTrend Airbender Pro NT610K...,Where can I feel my device?,[],[],[],[],[]
1291,electronics,tight,case,great,protection,ed6533390fe2fe0d546b69dea570c34b,5,0.00,False,ebaaec25661c0b21f3e5600a1ed7e30b,ee5906ed3306b7c6d219a40adae2c7c2,B00H3OYSHW,I ordered my new NewTrend Airbender Pro NT610K...,How does the selector protection me?,[],[],[],[],[]
1292,electronics,good,texture,good,grip,22b23446daf9d5bc0140be7278b220ee,1,0.00,False,84efbd9643d7f43af0ea0d2468c7dcc9,a541ffd9697ebf0ad9d226428594128e,B00HNGB1YS,"After using this case for a few days, I have g...",How is the grip?,[],[],[],[],[]
1293,electronics,easy to use,keyboard,removable,keyboard,e7967ad5be0658b4acfca1118f6af1ab,1,0.00,False,e856d7e5351e3385b5d75f06562e905b,7efc5e7ffaa79f53cc50f76a44f9f88e,B00HPM1G8Q,Edit 4/21/14. I finally caved and purchased a ...,How is the keyboard?,"[the keyboard is very easy to use, the keyboar...","[258, 258]","[1, 1]","[1.0, 1.0]","[True, True]"


In [22]:
for split, df in dfs.items():
    print(f"number of qs in {split}: {df['id'].nunique()}")

number of qs in train: 1295
number of qs in test: 358
number of qs in validation: 255


In [23]:
qa_cols = ["title", "question",
           "answers.text", "answers.answer_start",
           "context"]

### 1.2 Extracting Answers from Text

In [31]:
from transformers import AutoTokenizer

In [32]:
model_ckpt = "deepset/minilm-uncased-squad2"

In [33]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Downloading:   0%|          | 0.00/107 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/477 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [73]:
inputs = tokenizer(
    text = "how much music can this hold?",
    text_pair = "an mp3 is about 1mb/minute, about 6000 hours",
    return_tensors="pt"
)

In [74]:
inputs

{'input_ids': tensor([[  101,  2129,  2172,  2189,  2064,  2023,  2907,  1029,   102,  2019,
         23378,  2003,  2055,  1015, 14905,  1013,  3371,  1010,  2055, 25961,
          2847,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [48]:
inputs.input_ids.shape

torch.Size([1, 22])

In [49]:
tokenizer.decode(inputs["input_ids"][0])

'[CLS] how much music can this hold? [SEP] an mp3 is about 1mb / minute, about 6000 hours [SEP]'

In [51]:
import torch
from transformers import AutoModelForQuestionAnswering

In [52]:
model_ckpt

'deepset/minilm-uncased-squad2'

In [53]:
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)

Downloading:   0%|          | 0.00/127M [00:00<?, ?B/s]

In [54]:
with torch.no_grad():
    outputs = model(**inputs)

In [55]:
outputs

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-1.2935, -4.7204, -5.3716, -5.2326, -5.2625, -5.3654, -4.9590, -6.0611,
         -1.2935, -0.2044, -0.1700, -1.9096,  4.1928,  5.7759, -2.2901, -3.7464,
         -1.2213, -3.4085,  2.6456,  3.3822, -0.8894, -1.2935]]), end_logits=tensor([[-1.2809, -5.4610, -5.0965, -5.2911, -5.5386, -5.4595, -5.3126, -4.6202,
         -1.2809, -3.4540, -0.8159, -3.7237, -2.7964, -1.0135,  1.1267, -2.5709,
          5.6335,  0.0113, -3.9191,  0.5685,  5.2458, -1.2809]]), hidden_states=None, attentions=None)

In [67]:
outputs.start_logits

tensor([[-1.2935, -4.7204, -5.3716, -5.2326, -5.2625, -5.3654, -4.9590, -6.0611,
         -1.2935, -0.2044, -0.1700, -1.9096,  4.1928,  5.7759, -2.2901, -3.7464,
         -1.2213, -3.4085,  2.6456,  3.3822, -0.8894, -1.2935]])

In [68]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits

In [69]:
start_idx = torch.argmax(start_logits)

In [71]:
end_idx = torch.argmax(end_logits) + 1

In [72]:
end_idx

tensor(17)

In [76]:
answer_span = inputs["input_ids"][0][start_idx:end_idx]

In [77]:
answer = tokenizer.decode(answer_span)

In [78]:
answer

'1mb / minute'